In [1]:
import pandas as pd
import numpy as np

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("key/bluebookkey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': "https://bluebookai2020.firebaseio.com"
})


In [5]:
from firebase_admin import db

In [6]:
ref = db.reference("act/raw_total_score/")
df_act_score = ref.get()
df_act_score=pd.DataFrame(df_act_score).T.reset_index(drop="index")
print(df_act_score.shape)
df_act_score.head(1)

(10183, 10)


,composite,eng,math,pecentile,reading,science,testDate,testNum,userID,userName
0,25.0,25.0,33.0,80.0,19.0,23.0,122317,R01,384.0,Eunbi Lee


In [7]:
# data cleaning before datetime type
df_act_score.loc[df_act_score["testDate"] == '12/19/', "testDate"] ="71619" 
df_act_score["testDate"]=pd.to_datetime(df_act_score["testDate"], format='%m%d%y')

In [8]:
df_act_score.head()

,composite,eng,math,pecentile,reading,science,testDate,testNum,userID,userName
0,25.0,25.0,33.0,80.0,19.0,23.0,2017-12-23,R01,384.0,Eunbi Lee
1,32.0,30.0,36.0,98.0,33.0,29.0,2019-07-25,R01,8041.0,Steven Lee
2,29.0,28.0,32.0,94.0,29.0,27.0,2019-07-25,R01,8050.0,Gabriel Song
3,24.0,26.0,25.0,74.0,24.0,23.0,2019-07-25,R01,3100.0,Christina Kim
4,30.0,32.0,33.0,96.0,29.0,28.0,2019-07-25,R01,717.0,Keonho Roh


In [261]:
act=df_act_score.sort_values(by="testDate").copy()

In [264]:
act_pivot=pd.pivot_table(act, values='composite', index=['userID'], columns=['testDate'], aggfunc=np.sum)
act_pivot=act_pivot.astype(str)

In [275]:
tmp=[]
for row in range(act_pivot.shape[0]):
    tmp2=[]
    for col in range(act_pivot.shape[1]):
        if act_pivot.iloc[row,col] !="nan":
            tmp2.append(act_pivot.iloc[row,col])
    tmp.append(tmp2)

In [278]:
max_s= []
fir_s=[]
pro=[]
for num in range(len(tmp)):
    _max=0
    for num2 in range(len(tmp[num])): 
        first=float(tmp[num][0].split(".")[0])
        com=float(tmp[num][num2].split(".")[0])
        if com > _max:
            _max = com
        #last=tmp[num][len(tmp[num])-1].split(".")[0]
    pro.append(int(_max)-int(first))
    max_s.append(int(_max))
    fir_s.append(int(first))
pro=pd.DataFrame(pro)
pro.head(1)

,0
0,2


In [282]:
act_pivot["progress"] = pro[0].values
act_pivot['max'] = max_s
act_pivot["first"] =fir_s

In [290]:
act_pivot["percent"] = act_pivot["progress"] / act_pivot['first']

In [346]:
mm=[]
for n,m in enumerate(act_pivot["max"]):
    if act_pivot.loc[n,"progress"] != 0:
        mm.append(m)
pd.DataFrame(mm).mean()

0    30.236674
dtype: float64

In [347]:
fs=[]
for n,f in enumerate(act_pivot["first"]):
    if act_pivot.loc[n,"progress"] != 0:
        fs.append(f)
pd.DataFrame(fs).mean()

0    25.321962
dtype: float64

In [284]:
sp=[]
for s in act_pivot["progress"]:
    if s != 0:
        sp.append(s)

In [285]:
pd.DataFrame(sp).mean().round(0)

0    5.0
dtype: float64

In [293]:
pp=[]
for p in act_pivot["percent"]:
    if p != 0:
        pp.append(p)

In [296]:
pd.DataFrame(pp).mean().round(2)

0    0.24
dtype: float64

In [300]:
act_pivot=act_pivot.reset_index()
act_pivot["userID"] = act_pivot["userID"].astype(float)

In [301]:
act_pivot.sort_values(by="userID")[["userID","progress",'max','first','percent']].to_csv("act_progress.csv")

In [302]:
act_pivot.sort_values(by="userID")[["userID","progress",'max','first','percent']].head(3)

testDate,userID,progress,max,first,percent
0,0.0,2,26,24,0.083333
1,1.0,7,26,19,0.368421
73,2.0,1,13,12,0.083333


In [348]:
act_pivot.shape

(1107, 640)

In [308]:
ref = db.reference("sat/raw_total_score/")
df_sat_score = ref.get()
df_sat_score=pd.DataFrame(df_sat_score).T.reset_index(drop="index")
print(df_sat_score.shape)
df_sat_score.head(1)

(1144, 8)


,Math,Reading_Writing,first,last,testDate,test_num,total,userID
0,570.0,590,Shelly,Yi,4/2/18,2,1160,104


In [309]:
sat=df_sat_score.sort_values(by="testDate").copy()

In [312]:
sat["testDate"] = pd.to_datetime(sat["testDate"],format="%m/%d/%y")

In [316]:
sat_pivot=pd.pivot_table(sat, values='total', index=['userID'], columns=['testDate'], aggfunc=np.sum)
sat_pivot=sat_pivot.astype(str)
sat_pivot.head(3)

testDate,2016-07-26 00:00:00,2016-07-27 00:00:00,2016-07-28 00:00:00,2016-07-29 00:00:00,2016-08-31 00:00:00,2016-09-01 00:00:00,2016-09-06 00:00:00,2016-09-10 00:00:00,2016-10-12 00:00:00,2016-10-26 00:00:00,...,2019-11-23 00:00:00,2019-11-24 00:00:00,2019-11-25 00:00:00,2019-11-26 00:00:00,2019-11-27 00:00:00,2019-11-28 00:00:00,2019-11-29 00:00:00,2019-12-01 00:00:00,2019-12-03 00:00:00,2019-12-04 00:00:00
userID,,,,,,,,,,,,,,,,,,,,,
1000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,1480,nan,nan,nan,nan,nan,nan,nan,nan,nan
1003,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,1240,nan,nan,nan,nan,nan,nan,nan,nan,nan
1005,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,1380,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [318]:
tmp=[]
for row in range(sat_pivot.shape[0]):
    tmp2=[]
    for col in range(sat_pivot.shape[1]):
        if sat_pivot.iloc[row,col] !="nan":
            tmp2.append(sat_pivot.iloc[row,col])
    tmp.append(tmp2)

In [319]:
max_s= []
fir_s=[]
pro=[]
for num in range(len(tmp)):
    _max=0
    for num2 in range(len(tmp[num])): 
        first=float(tmp[num][0].split(".")[0])
        com=float(tmp[num][num2].split(".")[0])
        if com > _max:
            _max = com
        #last=tmp[num][len(tmp[num])-1].split(".")[0]
    pro.append(int(_max)-int(first))
    max_s.append(int(_max))
    fir_s.append(int(first))
pro=pd.DataFrame(pro)
pro.head(1)

,0
0,110


In [322]:
sat_pivot["progress"] = pro[0].values
sat_pivot['max'] = max_s
sat_pivot["first"] =fir_s
sat_pivot["percent"] = sat_pivot["progress"] / sat_pivot['first']

In [343]:
mm=[]
for n,m in enumerate(sat_pivot["max"]):
    if sat_pivot.loc[n,"progress"] != 0 and m < 1600:
        mm.append(m)
pd.DataFrame(mm).mean()

0    1351.0
dtype: float64

In [345]:
fs=[]
for n,f in enumerate(sat_pivot["first"]):
    if sat_pivot.loc[n,"progress"] != 0 and f < 1600:
        fs.append(f)
pd.DataFrame(fs).mean()

0    1208.133333
dtype: float64

In [326]:
sp=[]
for s in sat_pivot["progress"]:
    if s != 0 and s < 1600:
        sp.append(s)

In [327]:
pd.DataFrame(sp).mean().round(0)

0    146.0
dtype: float64

In [329]:
pp=[]
for p in sat_pivot["percent"]:
    if p != 0 and p < 1:
        pp.append(p)

In [333]:
pd.DataFrame(pp).mean().round(2)

0    0.14
dtype: float64

In [331]:
sat_pivot=sat_pivot.reset_index()
sat_pivot["userID"] = act_pivot["userID"].astype(float)
sat_pivot.sort_values(by="userID")[["userID","progress",'max','first','percent']].to_csv("sat_progress.csv")
sat_pivot.sort_values(by="userID")[["userID","progress",'max','first','percent']].head()

testDate,userID,progress,max,first,percent
0,0.0,110,1500,1390,0.079137
1,1.0,210,1240,1030,0.203883
73,2.0,110,1400,1290,0.085271
129,3.0,140,1170,1030,0.135922
257,4.0,0,1330,1330,0.000000
